In [1]:
import os
import torch
import random
import warnings
import numpy as np
import torchvision
import torch.nn as nn
from tqdm import tqdm
from copy import deepcopy
from kornia import augmentation
import torch.nn.functional as F
import matplotlib.pyplot as plt
from diffusers import UNet2DModel
from datasets import load_dataset
from torchvision import transforms
from diffusers import DDPMScheduler
from PIL import ImageDraw, ImageFont, Image
from datasets import load_dataset, load_metric
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings("ignore")

### Define Generator, Victim and Stolen

In [2]:
def get_generator(args):
    class Flatten(nn.Module):
        def __init__(self):
            super(Flatten, self).__init__()
    
        def forward(self, x):
            return torch.flatten(x, 1)
    
    
    class Generator(nn.Module):
        def __init__(self, nz=100, ngf=64, img_size=32, nc=3):
            super(Generator, self).__init__()
            self.params = (nz, ngf, img_size, nc)
            self.init_size = img_size // 4
            self.l1 = nn.Sequential(nn.Linear(nz, ngf * 2 * self.init_size ** 2))
    
            self.conv_blocks = nn.Sequential(
                nn.BatchNorm2d(ngf * 2),
                nn.Upsample(scale_factor=2),
    
                nn.Conv2d(ngf * 2, ngf * 2, 3, stride=1, padding=1, bias=False),
                nn.BatchNorm2d(ngf * 2),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Upsample(scale_factor=2),
    
                nn.Conv2d(ngf * 2, ngf, 3, stride=1, padding=1, bias=False),
                nn.BatchNorm2d(ngf),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(ngf, nc, 3, stride=1, padding=1),
                nn.Sigmoid(),
            )
    
        def forward(self, z):
            out = self.l1(z)
            out = out.view(out.shape[0], -1, self.init_size, self.init_size)
            img = self.conv_blocks(out)
            return img
    
        # return a copy of its own
        def clone(self):
            clone = Generator(self.params[0], self.params[1], self.params[2], self.params[3])
            clone.load_state_dict(self.state_dict())
            return clone.cuda()

    return Generator(nz=args.gen_dim_z, ngf=64, img_size=args.img_w, nc=args.img_c).to(args.device)

In [3]:
class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 32, 3, 1)
            self.conv2 = nn.Conv2d(32, 64, 3, 1)
            self.fc1 = nn.Linear(12544, 128)
            self.fc2 = nn.Linear(128, 10)
    
        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = F.relu(self.conv2(x))
            x = F.max_pool2d(x, 2)
            x = torch.flatten(x, 1)
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return x

In [4]:
class NetS(nn.Module):
        def __init__(self):
            super(NetS, self).__init__()
            self.conv1 = nn.Conv2d(1, 32, 3, 1)
            self.conv2 = nn.Conv2d(32, 64, 3, 1)
            self.fc1 = nn.Linear(12544, 128)
            self.fc2 = nn.Linear(128, 10)
            self.bn1 = nn.BatchNorm2d(32)
            self.bn2 = nn.BatchNorm2d(64)
    
        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = self.bn1(x)
            x = F.relu(self.conv2(x))
            x = self.bn2(x)
            x = F.max_pool2d(x, 2)
            x = torch.flatten(x, 1)
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return x

In [5]:
def get_victim_clone(args):
    
    student = NetS().to(args.device)
    victim = torch.load(args.victim_path, map_location=args.device)
    return victim , student

### Utilites

In [6]:
class Args:
    def __init__(self, device, seed, epochs, batch_size, img_n, img_c, momentum,
                 img_w, img_h, lr_G, lr_C, lr_hee, lr_z, weight_decay, N_G, N_C, 
                 steps_hee, grad_accumulation_steps, std_aug, lam, label_smooth_factor,
                 victim_path, N_classes, debug, gen_dim_z, save_dir):
        
        self.device = device
        self.seed = seed
        self.epochs = epochs
        self.batch_size = batch_size
        self.img_n = img_n
        self.img_c = img_c
        self.img_w = img_w
        self.img_h = img_h
        self.lr_G = lr_G
        self.lr_C = lr_C
        self.lr_hee = lr_hee
        self.lr_z = lr_z
        self.weight_decay = weight_decay
        self.momentum = momentum
        self.N_G = N_G
        self.N_C = N_C
        self.steps_hee = steps_hee
        self.grad_accumulation_steps = grad_accumulation_steps
        self.std_aug = std_aug
        self.lam = lam
        self.victim_path = victim_path
        self.N_classes = N_classes
        self.debug = debug
        self.victim_path = victim_path
        self.gen_dim_z = gen_dim_z
        self.label_smooth_factor = label_smooth_factor
        self.save_dir = save_dir

In [7]:
class FakeDataset(torch.utils.data.Dataset):
    """Some Information about FakeDataset"""

    def __init__(self, root="", transform=None):
        super(FakeDataset, self).__init__()

        self.transform = transform

        history_images = np.load(os.path.join(root, "fake_images.npy"))
        self.images = torch.from_numpy(history_images)

    def __getitem__(self, index):
        image = self.images[index]

        return image

    def __len__(self):
        return len(self.images)

In [8]:
class TensorDataset(Dataset):
    def __init__(self, tensor):
        self.tensor = tensor

    def __len__(self):
        return len(self.tensor)

    def __getitem__(self, idx):
        return self.tensor[idx]

In [9]:
class DataIter(object):
    def __init__(self, dataloader):
        self.dataloader = dataloader
        self._iter = iter(self.dataloader)

    def next(self):
        try:
            data = next(self._iter)
        except StopIteration:
            self._iter = iter(self.dataloader)
            data = next(self._iter)
        return data

In [10]:
def get_standard_augment(img_w, img_h):
    std_aug = augmentation.container.ImageSequential(
    augmentation.RandomCrop(size=[img_w, img_h], padding=4),
    augmentation.RandomHorizontalFlip(),
)
    return std_aug

In [11]:
def strong_aug(image):
    device = image.device
    image = TF.center_crop(
        image,
        [int(32.0 * random.uniform(0.95, 1.0)), int(32.0 * random.uniform(0.95, 1.0))],
    )
    image = TF.resize(image, [32, 32])
    noise = torch.randn_like(image).to(device) * 0.001
    image = torch.clamp(image + noise, 0.0, 1.0)
    if random.uniform(0, 1) > 0.5:
        image = TF.vflip(image)
    if random.uniform(0, 1) > 0.5:
        image = TF.hflip(image)
    angles = [-15, 0, 15]
    angle = random.choice(angles)
    image = TF.rotate(image, angle)
    return image

In [12]:
def smooth_one_hot(args, true_labels):
    """
    if smoothing == 0, it's one-hot method
    if 0 < smoothing < 1, it's smooth method
    """
    device = true_labels.device
    true_labels = torch.nn.functional.one_hot(true_labels, args.N_classes).detach().cpu()
    assert 0 <= args.label_smooth_factor < 1
    confidence = 1.0 - args.label_smooth_factor
    label_shape = torch.Size((true_labels.size(0), args.N_classes))
    with torch.no_grad():
        true_dist = torch.empty(size=label_shape, device=true_labels.device)
        true_dist.fill_(args.label_smooth_factor / (args.N_classes - 1))
        _, index = torch.max(true_labels, 1)

        true_dist.scatter_(1, torch.LongTensor(index.unsqueeze(1)), confidence)
    return true_dist.to(device)

In [13]:
def evaluate(net, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images.to(device))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.to(device)).sum().item()
    
    print(f'Accuracy of the network on the 10,000 test images: {100 * correct / total:.2f}%')


In [14]:
def save_batch_fake(args, images, epoch):
    images = images.detach().cpu().numpy()
    images_filename = os.path.join(args.save_dir, "fake_images.npy")

    if epoch > 1:
        org_images = np.load(images_filename)

        images = np.concatenate((org_images, images), 0)

    np.save(images_filename, images)

### DFME++ Attack

In [15]:
class Entropy_Loss(nn.Module):
    def __init__(self, reduction="mean"):
        super(Entropy_Loss, self).__init__()
        self.reduction = reduction

    def forward(self, x):
        b = F.softmax(x, dim=1) * F.log_softmax(x, dim=1)
        b = -1.0 * b.sum(dim=1)
        if self.reduction == "mean":
            return b.mean()
        elif self.reduction == "sum":
            return b.sum()
        elif self.reduction == "none":
            return b

In [16]:
def logodds(logits):
    eps = 1e-7
    softmax_odds = F.softmax(logits, dim=1)
    softmax_odds = torch.clamp(softmax_odds, min = eps, max = 1-eps)
    log_odd = torch.log((softmax_odds) / (1-softmax_odds) )
    log_sigmoid_odd = torch.nn.LogSigmoid()
    return -log_sigmoid_odd(log_odd).sum()

In [17]:
def cross_entropy(outputs, smooth_labels):
    loss = torch.nn.KLDivLoss(reduction="batchmean")
    return loss(F.log_softmax(outputs, dim=1), smooth_labels)

In [18]:
def generate_hee(args, model, x):
    model.eval()
    x_hee = x.detach() + 0.001 * torch.torch.randn(x.shape).to(args.device).detach()
    for _ in range(args.steps_hee):
        x_hee.requires_grad_()
        with torch.enable_grad():
            pred = model(x_hee)
            loss = Entropy_Loss(reduction="mean")(pred)
        grad = torch.autograd.grad(loss, [x_hee])[0] 
        x_hee = x_hee.detach() + args.lr_hee * torch.sign(grad.detach())
        x_hee = torch.clamp(x_hee, 0.0, 1.0)
    model.train()

    return x_hee

In [19]:
def train_generator(args, generator_model, clone_model, epoch):

    generator_model.train()
    clone_model.eval()

    best_fake = None
    best_loss = 1e6
    
    if args.debug: print('Debug(train_generator) :-> Generating Images using Generator Model')
    z = torch.randn(size=(args.img_n, args.gen_dim_z)).to(args.device)
    z.requires_grad = True
    
    optimizer_G = torch.optim.Adam([{"params": generator_model.parameters()}, {"params": [z], "lr": args.lr_z}], lr=args.lr_G, betas=[0.5, 0.999])
    pseudo_y = torch.randint(low=0, high=args.N_classes, size=(args.img_n,)).to(args.device)
    soft_labels = smooth_one_hot(args, pseudo_y)
    
    if args.debug: print('Debug(train_generator) :-> Starting Generator Training')
    for step in range(args.N_G):

        fake = generator_model(z)
        aug_fake = args.std_aug(fake)
        
        logits = clone_model(aug_fake)

        loss_cls = cross_entropy(logits, soft_labels)
        loss_logodd = logodds(logits)
        loss = loss_logodd * args.lam

        with torch.no_grad():
            if best_loss > loss.item() or best_fake is None:
                best_loss = loss.item()
                best_fake = fake
        
        optimizer_G.zero_grad()
        loss.backward()
        # torch.nn.utils.clip_grad_norm(clone_model.parameters(), max_norm = 1)
        optimizer_G.step()
    print(f"Generator Step {step} average loss: {loss}")
    save_batch_fake(args, best_fake.data, epoch)
    if args.debug: print('Debug(train_generator) :-> Generator Training Ended')
    return generator_model

In [20]:
def train_clone(args, generator_model, clone_model, victim_model, optimizer_C):

    generator_model.eval()
    clone_model.train()
    victim_model.eval()

    dataset = FakeDataset(root=args.save_dir)
    data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=args.batch_size,
        shuffle=True,
    )
    data_iter = DataIter(data_loader)

    victim_classes_fired = np.zeros(args.N_classes)
    clone_classes_fired = np.zeros(args.N_classes)
    losses = []
    if args.debug: print('Debug(train_clone) :-> Starting Clone Model Training')        
    for step in range(args.N_C):
        fake = data_iter.next().to(args.device)
        aug_fake = args.std_aug(fake)
        fake_hee = generate_hee(args, clone_model,strong_aug(aug_fake))
        logits_T = victim_model(fake_hee).detach() 
        hard_labels = logits_T.topk(1, 1)[1].reshape(-1)
        np.add.at(victim_classes_fired, hard_labels.cpu().numpy() , 1)
        logits_C = clone_model(fake_hee)
        np.add.at(clone_classes_fired, logits_C.topk(1, 1)[1].reshape(-1).cpu().numpy() , 1)
        
        loss = F.cross_entropy(logits_C, hard_labels)
        losses.append(loss.item())
        optimizer_C.zero_grad()
        loss.backward()
        optimizer_C.step()
        if step % 200 == 0:
            print(f"Clone Steps {step} average loss: {sum(losses[-len(data_loader):])/len(data_loader)}")
            print(f"Clone Steps {step} Victim Classes Fired: {victim_classes_fired})")
            print(f"Clone Steps {step} Clone Classes Fired: {clone_classes_fired})")
    if args.debug: print('Debug(train_clone) :-> Clone Model Training Ended') 
    return (generator_model, clone_model, victim_model, optimizer_C)

In [21]:
debug = 1 #To debug code
device = torch.device('cuda') #device placement cpu or gpu
seed = 10 #seed for consistent result
epochs = 10 #number of epochs to train
batch_size = 256 #per device batch size
img_n = 256 #min(160, batch_size*10*2) #per epoch image generation count
img_c = 1 #image channel
img_w = 32 #image size
img_h = 32 #image size
lr_z = 0.01 #learning rate of latent code
lr_G = 0.01 #learning rate of Generator
lr_C = 0.09 #learing rate of clone model
lr_hee = 0.03 #perturb number of steps
weight_decay = 1e-4 #Optimizer parameter: decay's weight update
momentum = 0.9 #Optimizer parameter: Remeber past information 1/momentum times
N_G = 10 #Diffuser train steps
N_C = 500 #Clone model steps 
steps_hee = 10 #number of epochs to train
grad_accumulation_steps = 16 #update model after no.of steps
std_aug = get_standard_augment(img_w, img_h) #standard augmentation: flip, crop
lam = 0.009 #hyperparameter for balancing two loss terms in diffuser
victim_path = "HEE/fashion_mnist.pt" #clone model path
N_classes = 10 #No.of classes to predict
gen_dim_z = 256 #Dimension of generator input noise
label_smooth_factor = 0.2 #Hard to soft logits
save_dir = r'Gen_Imgs\LogOdds' #Folder to save genrator images

In [22]:
args = Args(
        debug = debug,
        device = device,
        seed = seed,
        epochs = epochs,
        batch_size = batch_size,
        img_n = img_n,
        img_c = img_c,
        img_w = img_w,
        img_h = img_h,
        lr_G = lr_G,
        lr_C = lr_C,
        lr_z = lr_z,
        lr_hee = lr_hee,
        weight_decay = weight_decay,
        momentum = momentum,
        N_G = N_G,
        N_C = N_C,
        steps_hee = steps_hee,
        grad_accumulation_steps = grad_accumulation_steps,
        std_aug = std_aug,
        lam = lam,
        victim_path = victim_path,
        N_classes = N_classes,
        label_smooth_factor = label_smooth_factor,
        gen_dim_z = gen_dim_z,
        save_dir = save_dir
    )


In [23]:
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [24]:
generator_model =  get_generator(args)
victim_model, clone_model = get_victim_clone(args)

In [25]:
optimizer_C = torch.optim.AdamW(
        clone_model.parameters(),
        lr=args.lr_C,
        weight_decay=args.weight_decay,
    )
scheduler_lr = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer_C, args.epochs, eta_min=2e-4
)
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((32, 32))])
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

In [ ]:
evaluate(victim_model, testloader), evaluate(clone_model, testloader)

In [26]:
 for epoch in tqdm(range(1, args.epochs + 1)):
     
    generator_model = train_generator(args, generator_model, clone_model, epoch)
    print('-'*50)
    generator_model, clone_model, victim_model, optimizer_C = train_clone(args, generator_model, clone_model, victim_model, optimizer_C)
    print('-'*50)
    print(scheduler_lr.get_last_lr())
    print()
    evaluate(clone_model, testloader)
    print()
    scheduler_lr.step()


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Debug(train_generator) :-> Generating Images using Generator Model
Debug(train_generator) :-> Starting Generator Training
Generator Step 9 average loss: 53.08906555175781
Debug(train_generator) :-> Generator Training Ended
--------------------------------------------------
Debug(train_clone) :-> Starting Clone Model Training
Clone Steps 0 average loss: 2.0337328910827637
Clone Steps 0 Victim Classes Fired: [ 10.   0.   1.   1.   0.   2.   4.   0. 238.   0.])
Clone Steps 0 Clone Classes Fired: [  5.   2.   4. 217.   0.   0.   0.   2.  26.   0.])
Clone Steps 200 average loss: 1.465746521949768
Clone Steps 200 Victim Classes Fired: [ 6122.   160.   290.  1520.    49. 11495.   967.  1661. 28425.   767.])
Clone Steps 200 Clone Classes Fired: [1.4100e+02 3.0000e+00 6.8000e+01 2.2500e+02 0.0000e+00 2.4500e+02
 2.1000e+01 7.0000e+00 5.0746e+04 0.0000e+00])
Clone Steps 400 average loss: 1.4149314165115356
Clone Steps 400 Victim Classes Fired: [12011.   336.   578.  3075.    91. 23045.  1980.  3

 10%|████████▎                                                                          | 1/10 [01:37<14:40, 97.83s/it]

Accuracy of the network on the 10,000 test images: 10.03%

Debug(train_generator) :-> Generating Images using Generator Model
Debug(train_generator) :-> Starting Generator Training
Generator Step 9 average loss: 83.86058807373047
Debug(train_generator) :-> Generator Training Ended
--------------------------------------------------
Debug(train_clone) :-> Starting Clone Model Training
Clone Steps 0 average loss: 0.5528713464736938
Clone Steps 0 Victim Classes Fired: [ 37.   0.   0.   4.   1.  28.   4.   5. 176.   1.])
Clone Steps 0 Clone Classes Fired: [  0.   0.   0.   0.   0.   0.   0.   0. 256.   0.])
Clone Steps 200 average loss: 1.3807624578475952
Clone Steps 200 Victim Classes Fired: [ 5780.   166.   342.  1490.    63. 11964.  1157.  1268. 28411.   815.])
Clone Steps 200 Clone Classes Fired: [    0.     0.     0.     0.     0.     0.     0.     0. 51456.     0.])
Clone Steps 400 average loss: 1.3626468777656555
Clone Steps 400 Victim Classes Fired: [11461.   329.   636.  2847.   10

 20%|████████████████▌                                                                  | 2/10 [03:16<13:04, 98.12s/it]

...

Accuracy of the network on the 10,000 test images: 9.89%

Debug(train_generator) :-> Generating Images using Generator Model
Debug(train_generator) :-> Starting Generator Training
Generator Step 9 average loss: 86.90513610839844
Debug(train_generator) :-> Generator Training Ended
--------------------------------------------------
Debug(train_clone) :-> Starting Clone Model Training
Clone Steps 0 average loss: 0.13998228973812527
Clone Steps 0 Victim Classes Fired: [ 22.   1.   0.   4.   1.  68.   4.   4. 144.   8.])
Clone Steps 0 Clone Classes Fired: [  0.   0.   0.   0.   0.   0.   0.   0. 256.   0.])
Clone Steps 200 average loss: 1.271870454152425
Clone Steps 200 Victim Classes Fired: [ 5557.   152.   368.  1500.    51. 11479.  1202.  1205. 29277.   665.])
Clone Steps 200 Clone Classes Fired: [    0.     0.     0.     0.     0.     0.     0.     0. 51456.     0.])
Clone Steps 400 average loss: 1.2563029395209417
Clone Steps 400 Victim Classes Fired: [10781.   278.   726.  2844.    91

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [14:44<01:38, 98.37s/it]

Accuracy of the network on the 10,000 test images: 9.89%

Debug(train_generator) :-> Generating Images using Generator Model
Debug(train_generator) :-> Starting Generator Training
Generator Step 9 average loss: 87.5033187866211
Debug(train_generator) :-> Generator Training Ended
--------------------------------------------------
Debug(train_clone) :-> Starting Clone Model Training
Clone Steps 0 average loss: 0.11843793392181397
Clone Steps 0 Victim Classes Fired: [ 13.   1.   3.   8.   0.  65.   4.   6. 155.   1.])
Clone Steps 0 Clone Classes Fired: [  0.   0.   0.   0.   0.   0.   0.   0. 256.   0.])
Clone Steps 200 average loss: 1.2895360708236694
Clone Steps 200 Victim Classes Fired: [ 5279.   159.   323.  1319.    48. 11674.  1151.  1142. 29590.   771.])
Clone Steps 200 Clone Classes Fired: [    0.     0.     0.     0.     0.     0.     0.     0. 51456.     0.])
Clone Steps 400 average loss: 1.2876222610473633
Clone Steps 400 Victim Classes Fired: [10719.   298.   678.  2666.    91

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [16:23<00:00, 98.35s/it]

Accuracy of the network on the 10,000 test images: 9.89%



In [ ]:
z = torch.randn(100,256).to(device)

In [ ]:
with torch.no_grad():
    fake = generator_model(z)

In [ ]:
plt.imshow(fake[52].cpu().numpy().transpose(1,2,0))

In [ ]:
logit = clone_model(fake)

In [ ]:
fake[0].max()

In [ ]:
victim_model(fake).topk(1,1)[1]